In [1]:
import pandas as pd
import requests
# import re
import os
# import pymongo
from bs4 import BeautifulSoup as bs
from splinter import Browser


In [2]:
browser = Browser("chrome", executable_path="chromedriver", headless=False)

In [3]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)

In [4]:
html=browser.html
soup = bs(html, "html.parser")

In [5]:
listing_title = soup.find_all("div", {"class": "content_title"})

In [6]:
len(listing_title)

52

In [7]:
news_title = listing_title[0].find('a').get_text() #will grab top news title
news_title

'Mars 2020 Stands on Its Own Six Wheels'

In [8]:
news_p = soup.find("div", class_="article_teaser_body").text
news_p

"In time-lapse video, taken at JPL, captures the first time NASA's Mars 2020 rover carries its full weight on its legs and wheels."

In [9]:
images_url ="https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(images_url)

In [12]:
base_url="https://www.jpl.nasa.gov"
browser.find_by_id('full_image').click()

In [13]:
browser.is_element_present_by_text("more info", wait_time=5)
more_info_element = browser.click_link_by_partial_text("more info")

In [14]:
html=browser.html
soup = bs(html, "html.parser")

In [15]:
other_url = soup.find("img", class_="main_image")["src"]
other_url

'/spaceimages/images/largesize/PIA13911_hires.jpg'

In [16]:
featured_url= base_url + other_url
featured_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA13911_hires.jpg'

In [17]:
twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(twitter_url)

In [18]:
html=browser.html
soup = bs(html, "html.parser")

In [19]:
mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
mars_weather

'InSight sol 321 (2019-10-22) low -102.9ºC (-153.2ºF) high -24.6ºC (-12.3ºF)\nwinds from the SSE at 4.8 m/s (10.8 mph) gusting to 18.6 m/s (41.6 mph)\npressure at 7.10 hPapic.twitter.com/FsE4d1Fskt'

In [20]:
marsfacts_url = "https://space-facts.com/mars/"
browser.visit(marsfacts_url)

In [21]:
html=browser.html
soup = bs(html, "html.parser")

In [22]:
marsfacts = pd.read_html(marsfacts_url)
mars_df = marsfacts[0]
mars_df.columns=['Description', 'Mars',"Earth" ]
mars_df.set_index('Description', inplace=True)
mars_df


,Mars,Earth
Description,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-153 to 20 °C,-88 to 58°C


In [23]:
df_mars = mars_df[['Mars']]
df_mars

,Mars
Description,
Diameter:,"6,779 km"
Mass:,6.39 × 10^23 kg
Moons:,2
Distance from Sun:,"227,943,824 km"
Length of Year:,687 Earth days
Temperature:,-153 to 20 °C


In [24]:
df_mars.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-153 to 20 °C</td>\n    </tr>\n  </tbody>\n</table>'

In [25]:
USGS_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(USGS_url)

In [26]:
html=browser.html
soup = bs(html, "html.parser")

In [27]:
# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = bs( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]